In [1]:
'''here, we're using GridSearchcv to tune the hyperparameters of some linear models and then return the best model and it's hyperparameters.'''
import sys
# sys.path.insert(0,"/Users/angelicaaluo/Airbnb/AIRBNB-DATASET/")
import pandas  as pd
import numpy as np
import os
import joblib
import json
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn. ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
# from tabular_data import load_airbnb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

df= pd.read_csv('/Users/angelicaaluo/Airbnb/AIRBNB-DATASET/airbnb-property-listings/tabular_data/clean_data.csv')
df.drop('Unnamed: 19',axis=1,inplace=True)  
# X,y = load_airbnb(df,"Price_Night")
X = df.select_dtypes(include=['int','float'])
y=df["Price_Night"]
mx=MinMaxScaler()
mx.fit(X,y)
model_hyperparam_distribution={
LinearRegression: {
    'fit_intercept': [True, False],
    # 'normalize': [True, False],
    'copy_X': [True, False],
    'n_jobs': [None, 1, 2, 3, 4, 5],
    # 'random_state': [42, 56, 71, 93]
},

DecisionTreeRegressor:{
    'max_depth': [3,5,7,10],
    'min_samples_leaf': [10, 20, 30, 50],
    'min_samples_split': [2, 5, 10, 20],
    'max_leaf_nodes': [5, 10, 15, 20],
    'max_features': [1.0, 'sqrt', 'log2', None],
    # 'criterion': ['mse', 'mae'],
    # 'criterion' :['poisson', 'squared_error', 'absolute_error', 'friedman_mse'],
    'splitter': ['best', 'random'],
    'random_state': [42, 56, 71, 93]
},

GradientBoostingRegressor:{
    'learning_rate': [0.01, 0.1, 0.5, 1],
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [3, 5, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 20],
    #'max_leaf_nodes': [5,15, 20, 25],
    'max_features': [ 'sqrt', 'log2', None],
    #'loss': ['absolute_error', 'squared_error', 'huber', 'quantile'],
    #'random_state': [42, 56, 71, 93]
},

SGDRegressor:{
    'loss': ['squared_error', 'epsilon_insensitive', 'huber', 'squared_epsilon_insensitive'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1],
    'l1_ratio': [0.1, 0.3, 0.5,0.9],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'max_iter': [250, 750, 500, 1000]
}

}

def tune_regression_model_hyperparameters(features, label, model_hyperparam_distribution):
    features = X
    label=y
    for md,hp in model_hyperparam_distribution.items():
        model = md()
        hyper_param=hp
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
        grid_search =GridSearchCV(estimator=model,param_grid=hp,verbose=10,refit=True)
        grid_search.fit(X_train,y_train) 
        tuned_model=grid_search.best_estimator_
        tuned_parameters=grid_search.best_params_
        eval_metrics={'MSE: ':mean_squared_error,
        'MAE ':mean_absolute_error,
        'r2 score: ':r2_score,
        # 'validation_RMSE: ': np.sqrt(mean_squared_error)
        }
        for name, metric in eval_metrics.items():
            y_pred = grid_search.predict(X_test)
            metric_value = metric(y_test, y_pred)
            metrics=(f'{name}: {metric_value}')
            print(metrics)
        destination='/Users/angelicaaluo/Airbnb/AIRBNB-DATASET/Linear_Regression /models/'
        if not os.path.isdir(destination):
            os.mkdir(destination)
        model_destination = f'{destination}/GridSearchcv'
        if not os.path.isdir(model_destination):
            os.mkdir(model_destination)
        model_final_path=f'{model_destination}/{model}'
        save_model(tuned_model,tuned_parameters,metrics,model_final_path)

    return tuned_model,tuned_parameters,metrics

def save_model(model,hyper_parameters,metrics,folder):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    model_path = f"{folder}/model.joblib"
    joblib.dump(model,model_path)

    hyperparameter_path = f'{folder}/hyperparameter.json'
    with open (hyperparameter_path, 'w') as hp:
        json.dump(str(hyper_parameters),hp)

    eval_path = folder + '/metrics.json'
    with open (eval_path, 'w') as ep:
        json.dump(str(metrics),ep)

def find_best_model():
    all_metrics_files = list(Path('./models/GridSearchcv/').glob('**/*metrics.json'))
    all_metric_data=[]
    for metric_file in all_metrics_files:
        with open(metric_file,'r') as f:
            metric_data=json.load(f)
            all_metric_data.append(metric_data)
    best_score=max(all_metric_data)
    for score in all_metric_data:
        if score==best_score:
            best_tuned_model=str(metric_file).split('/')[-2]
            best_tuned_model_path= f'./models/GridSearchcv/{best_tuned_model}'
            best_model_name=joblib.load(f'{best_tuned_model_path}/model.joblib')
            with open(f'{best_tuned_model_path}/hyperparameter.json', 'r') as hp:
                best_model_hp=json.load(hp)
            with open(f'{best_tuned_model_path}/metrics.json', 'r') as score:
                best_model_score=json.load(score)
            return best_model_name,best_model_hp,best_model_score



tune_regression_model_hyperparameters(X,y,model_hyperparam_distribution)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5; 1/24] START copy_X=True, fit_intercept=True, n_jobs=None...............
[CV 1/5; 1/24] END copy_X=True, fit_intercept=True, n_jobs=None;, score=1.000 total time=   0.1s
[CV 2/5; 1/24] START copy_X=True, fit_intercept=True, n_jobs=None...............
[CV 2/5; 1/24] END copy_X=True, fit_intercept=True, n_jobs=None;, score=1.000 total time=   0.0s
[CV 3/5; 1/24] START copy_X=True, fit_intercept=True, n_jobs=None...............
[CV 3/5; 1/24] END copy_X=True, fit_intercept=True, n_jobs=None;, score=1.000 total time=   0.0s
[CV 4/5; 1/24] START copy_X=True, fit_intercept=True, n_jobs=None...............
[CV 4/5; 1/24] END copy_X=True, fit_intercept=True, n_jobs=None;, score=1.000 total time=   0.0s
[CV 5/5; 1/24] START copy_X=True, fit_intercept=True, n_jobs=None...............
[CV 5/5; 1/24] END copy_X=True, fit_intercept=True, n_jobs=None;, score=1.000 total time=   0.0s
[CV 1/5; 2/24] START copy_X=True, fit_intercept=